# Introduction
## Importing Libraries

In [1]:
print('[-] Importing packages...')
# File Connection and File Manipulation
import os
import pickle
import gzip
import json
# Basic Data Science Toolkits
import pandas as pd
import numpy as np
import math
import random
import time
import re
import itertools
import datetime
# Basic Data Vizualization
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
# Text Preprocessing(other)
import string
import nltk

[-] Importing packages...


In [2]:
print('[-] Importing packages...')
try:
  import pyvis
  from pyvis.network import Network
except:
  !pip install pyvis
  import pyvis
  from pyvis.network import Network

[-] Importing packages...


In [3]:
#Download required corpus based data to nltk package
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\silve\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\silve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\silve\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\silve\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Set Global Variables

In [4]:
gDEBUG = True

## Verify GPU runtime

In [5]:
#see the GPU assigned
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Apr  1 16:43:54 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.52                 Driver Version: 551.52         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   58C    P5             18W /  115W |    2664MiB /   8192MiB |     27%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Set Up Directories

In [23]:
## Mount Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
ROOT_DIR = "./"
DATA_DIR = "%s/data" % ROOT_DIR
EVAL_DIR = "%s/evaluation" % ROOT_DIR
MODEL_DIR = "%s/models" % ROOT_DIR

#Create missing directories, if they don't exist
if not os.path.exists(DATA_DIR):
    # Create a new directory because it does not exist
    os.makedirs(DATA_DIR)
    print("The data directory is created!")
if not os.path.exists(EVAL_DIR):
    # Create a new directory because it does not exist
    os.makedirs(EVAL_DIR)
    print("The evaluation directory is created!")
if not os.path.exists(MODEL_DIR):
    # Create a new directory because it does not exist
    os.makedirs(MODEL_DIR)
    print("The model directory is created!")

The data directory is created!
The evaluation directory is created!
The model directory is created!


# Loading the Data
## Downloading the Data

In [7]:
#link would be expired by the time of the grading. It's just here to show how it could be done.
#import urllib.request
#url = 'https://d3c33hcgiwev3.cloudfront.net/Rylguh2DR_CpYLodg0fwQQ_1199ff1bd63f4163bc4de1e7806c70f1_nikelululemonadidas_tweets.jsonl.gz?Expires=1712102400&Signature=i9uv4cAI6bDpP2tP3nODY~qmBSPEt~ywYmy6H0i9CIjG2siek56an3vNbEzUC8BESaVtb-sjqQAALUtjHeyTEANAs8dnUXs~vISoeH7FydUKZYO5rJAjO2Xj~NxYRBt1e78rUILA4ZPP2LWWbrvxpOBW-yNBWu19KAH8nk74DrM_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'
#filename = './data/nikelululemonadidas_tweets.jsonl.gz'
#urllib.request.urlretrieve(url, filename)

## Basic Inspection and Sanity Check

In [13]:
LIMIT = 5
tweet_file_path = '%s/nikelululemonadidas_tweets.jsonl.gz' % DATA_DIR
with gzip.open(tweet_file_path) as f:
    for i, line in enumerate(f):
        if i >= LIMIT:
            break
        tweet = json.loads(line)
        text = tweet.get("full_text") or tweet.get("text")
        if "nike" in text.lower():
            print('tweet', i,':', text, '\n')

tweet 0 : #ad The Nike Women's Air More Uptempo 96 'White/Opti Yellow' is now available via @footlocker! |$160| #SneakerScouts @Nike https://t.co/5lAq7b2ffU https://t.co/wmjxIcsheP 

tweet 2 : Proof @LaserShip is stealing. I work from home and have a ring doorbell. @wsoctv @Nike @wcnc @wbtv @bbb_us https://t.co/9o3stezjgs 

tweet 3 : RT @pyleaks: *LEAK ALERT*: The next Supreme x @Nike collab for Spring 2022 will feature the Nike Shox Ride 2.
The duo will be dropping 3 co… 

tweet 4 : RT @SneakerScouts: #ad The Space Jam x Nike LeBron 18 Low 'Sylvester vs. Tweety' is now available via @snipes_usa! |$160| #SneakerScouts @K… 



In [14]:
tweet

{'created_at': 'Fri Oct 01 06:22:08 +0000 2021',
 'id': 1443823529296867300,
 'id_str': '1443823529296867350',
 'full_text': "RT @SneakerScouts: #ad The Space Jam x Nike LeBron 18 Low 'Sylvester vs. Tweety' is now available via @snipes_usa! |$160| #SneakerScouts @K…",
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [{'text': 'ad', 'indices': [19, 22]},
   {'text': 'SneakerScouts', 'indices': [122, 136]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'SneakerScouts',
    'name': '#SneakerScouts',
    'id': 792182745942233100,
    'id_str': '792182745942233089',
    'indices': [3, 17]},
   {'screen_name': 'snipes_usa',
    'name': 'SNIPES_USA',
    'id': 83606701,
    'id_str': '83606701',
    'indices': [102, 113]}],
  'urls': []},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 

# Create a Mention Network
## Load the Corpus of Tweets

In [24]:
## Load the Product Data
##this assigns the filename we're trying to load in to a string variable
tweet_file_path = '%s/nikelululemonadidas_tweets.jsonl.gz' % DATA_DIR
json_file = gzip.open(tweet_file_path, 'r')

## Create a List of Unique Users

In [17]:
# Identify unique users in the mention network
unique_users = {}

#iterate through each row in the twitter corpus and extract twitter users
for i, atweet in enumerate(json_file):
    if i % 10000 == 0: # Show a periodic status
      print("%s tweets iterated" % i)
    tweet_json = json.loads(atweet)
    user_who_tweeted = tweet_json['user']['screen_name']
    id_who_tweeted = tweet_json['user']['id']
    follower_count = tweet_json['user']['followers_count']
    if id_who_tweeted in unique_users:
      unique_users[id_who_tweeted]['tweet_count'] += 1
      if unique_users[id_who_tweeted]['followers_count'] == 0:
        unique_users[id_who_tweeted]['followers_count'] = follower_count
    if id_who_tweeted not in unique_users:
      unique_users[id_who_tweeted] = {}
      unique_users[id_who_tweeted]['tweet_count'] = 1
      unique_users[id_who_tweeted]['mention_count'] = 0
      unique_users[id_who_tweeted]['id'] = id_who_tweeted
      unique_users[id_who_tweeted]['followers_count'] = follower_count
      unique_users[id_who_tweeted]['screen_name'] = user_who_tweeted.lower()
    #add in the mentioned users as well
    users_mentioned = tweet_json['entities']['user_mentions']
    #if the tweet netions other users in the tweet
    if len(users_mentioned) > 0:
      # iterate through each mention in the tweet
      for user_mentioned in users_mentioned:
        #extract details about user
        screen_name_mentioned = user_mentioned['screen_name']
        id_mentioned = user_mentioned['id']
        #if in the list, increase the mention count
        if id_mentioned in unique_users:
          unique_users[id_mentioned]['mention_count'] += 1
        #extract details about the metioned users in the tweet
        if id_mentioned not in unique_users:
          unique_users[id_mentioned] = {}
          unique_users[id_mentioned]['tweet_count'] = 0
          unique_users[id_mentioned]['mention_count'] = 1
          unique_users[id_mentioned]['id'] = id_mentioned
          unique_users[id_mentioned]['followers_count'] = 0
          unique_users[id_mentioned]['screen_name'] = screen_name_mentioned.lower()

print(f"{i} total Tweets processed")

0 tweets iterated
10000 tweets iterated
20000 tweets iterated
30000 tweets iterated
40000 tweets iterated
50000 tweets iterated
60000 tweets iterated
70000 tweets iterated
80000 tweets iterated
90000 tweets iterated
100000 tweets iterated
110000 tweets iterated
120000 tweets iterated
130000 tweets iterated
140000 tweets iterated
150000 tweets iterated
160000 tweets iterated
170000 tweets iterated
175077 total Tweets processed


In [18]:
#print the number of unique users in the corpus
print('there are', len(unique_users), 'total users in the mention network.')

there are 131663 total users in the mention network.


## Create User Filter List

In [19]:
users_to_include = set()
# if any user has more than 2 tweets
min_tweet_count = 2
# if any user has more than 2500 followers
min_followers_count = 100000
# create a list of brand users
brand_users = ['nike', 'lululemon', 'adidas']

# a set is a unique list
user_count = 0

#iterate through the list of unique users and filter down the list
for auser in unique_users:
  if unique_users[auser]['screen_name'] in brand_users:
      print('id:', auser, '\tscreen_name:', unique_users[auser]['screen_name'])
      user_count += 1
      users_to_include.add(auser)
  elif unique_users[auser]['tweet_count'] >= min_tweet_count:
    if unique_users[auser]['followers_count'] >= min_followers_count:
      user_count += 1
      users_to_include.add(auser)
print(len(users_to_include))

id: 415859364 	screen_name: nike
id: 300114634 	screen_name: adidas
id: 16252784 	screen_name: lululemon
198


In [20]:
#print the number of unique users to include in the graph
print('there are', len(users_to_include), 'users to include in the final mention network.')
pct_of_total = round(len(users_to_include) / len(unique_users),4) * 100
print(f"{pct_of_total}% of the total users are included in the mention network.")

there are 198 users to include in the final mention network.
0.15% of the total users are included in the mention network.


In [21]:
#print the word counts for the major brands
print("Nike:", unique_users[415859364])
print("Adidas:", unique_users[300114634])
print("Lululemon:", unique_users[16252784])

Nike: {'tweet_count': 0, 'mention_count': 120125, 'id': 415859364, 'followers_count': 0, 'screen_name': 'nike'}
Adidas: {'tweet_count': 3, 'mention_count': 36654, 'id': 300114634, 'followers_count': 4082910, 'screen_name': 'adidas'}
Lululemon: {'tweet_count': 0, 'mention_count': 6294, 'id': 16252784, 'followers_count': 0, 'screen_name': 'lululemon'}


## Setting Up Helper Functions for Analysis

In [25]:
def graph_summary_stats(G, title='Graph Summary'):
  #display a summary of the graph object created
  #nx.info(G)
  # https://networkx.org/documentation/stable/reference/functions.html
  print('----------------------------------------')
  print('#####', title, '#####')
  print('number of nodes:', nx.number_of_nodes(G))
  print('number of edges:', nx.number_of_edges(G))
  print()
  print('nodes:', nx.nodes(G))
  print()
  if G.has_node('adidas'):
    print('neighbors of adidas:', list(nx.all_neighbors(G, 'adidas')))
  if G.has_node('nike'):
    print('neighbors of nike:', list(nx.all_neighbors(G, 'nike')))
  if G.has_node('lululemon'):
   print('neighbors of lululemon:', list(nx.all_neighbors(G, 'lululemon')))
  print('----------------------------------------\n')

In [26]:
#function to plot the graph object and write to png
def plot_graph(G, file_path='temp_file', use_edge_weight=True, plot_size='large'):
  #define node colors
  default_color = 'blue'
  highlight_color = 'red'
  brand_users = ['nike', 'lululemon', 'adidas']
  node_colors = [highlight_color if node in brand_users else default_color for node in G.nodes()]

  #set plot size for drawing
  if plot_size == 'medium-large':
    p_figsize = (150, 150)
    p_font_size = 20
    p_edge_width_scale = 2
    p_node_size = 5000
    p_arrow_size = 50
    p_k = None
  if plot_size == 'medium':
    p_figsize = (25, 25)
    p_font_size = 12
    p_edge_width_scale = 2
    p_node_size = 3000
    p_arrow_size = 50
    p_k = None
  elif plot_size == 'small':
    p_figsize = (50, 50)
    p_font_size = 20
    p_edge_width_scale = 2
    p_node_size = 30000
    p_arrow_size = 100
    p_k = None
  elif plot_size == 'x-small':
    p_figsize = (12, 12)
    p_font_size = 20
    p_edge_width_scale = 2
    p_node_size = 5000
    p_arrow_size = 100
    p_k = None
  else: #plot_size == 'large'
    p_figsize = (300, 300)
    p_font_size = 20
    p_edge_width_scale = 2
    p_node_size = 3000
    p_arrow_size = 100
    p_k = None

  #generate a spring layout for the graph
  positions = nx.spring_layout(G, k=p_k)

  #extract edge weights for drawing
  if use_edge_weight == True:
    p_edge_weights = p_edge_width_scale*[G[u][v]['weight'] for u, v in G.edges()]
  else:
    p_edge_weights = p_edge_width_scale

  #create a figure and adjust the size of the figure
  fig, ax = plt.subplots(1, 1, figsize=p_figsize)
  #draw the graph of the network
  nx.draw_networkx(G, pos=positions, ax=ax, node_color=node_colors,
                   font_color="#FFFFFF", font_size=p_font_size,
                   node_size=p_node_size, width=p_edge_weights,
                   arrows=True, arrowsize=p_arrow_size)
  #save the graph to a file
  print_file_path = '%s/%s.png' % (MODEL_DIR,file_path)
  plt.savefig(print_file_path, format='PNG')
  plt.close('all')

In [27]:
def focus_edges(G, brand_nodes = None, weight_min = None, weight_max = None):
  # Filter based on a list of brand nodes to focus
  if brand_nodes != None:
    # Filter edges based on the weight threshold
    filtered_edges = [(u, v) for u, v in G.edges() if u in brand_nodes or v in brand_nodes]
    # Create a subgraph based on the filtered edges
    G = G.edge_subgraph(filtered_edges)
  # Filter based on weight threshold
  if weight_min != None:
    # Filter edges based on the weight threshold
    filtered_edges = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] >= weight_min]
    # Create a subgraph based on the filtered edges
    G = G.edge_subgraph(filtered_edges)
  if weight_max != None:
    # Filter edges based on the weight threshold
    filtered_edges = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] <= weight_max]
    # Create a subgraph based on the filtered edges
    G = G.edge_subgraph(filtered_edges)
  # Return the filtered subgraph
  return G

## Create a Graph Object

In [28]:
## Load the Product Data
##this assigns the filename we're trying to load in to a string variable
tweet_file_path = '%s/nikelululemonadidas_tweets.jsonl.gz' % DATA_DIR
json_file = gzip.open(tweet_file_path, 'r')

In [29]:
#create a blank graph object
MEN_Graph = nx.DiGraph()

In [30]:
# Identify unique users in the mention network
for i, atweet in enumerate(json_file):
    #provide an update on processing progress
    if i % 10000 == 0:
      print("%s tweets iterated" % i)
    #load a line of the twitter corpus
    tweet_json = json.loads(atweet)
    #extract details from the tweet
    user_who_tweeted = tweet_json['user']['screen_name'].lower()
    id_who_tweeted = tweet_json['user']['id']
    follower_count = tweet_json['user']['followers_count']
    #check if the user id is in the list to include
    if id_who_tweeted in users_to_include:
      users = tweet_json['entities']['user_mentions']
      #if the tweet netions other users in the tweet
      if len(users) > 0:
          # iterate through each mention in the tweet
          for auser in users:
              #extract details about the metioned users in the tweet
              screen_name = auser['screen_name'].lower()
              mention_id = auser['id']
              #if the mentioned user is on the list to included
              if mention_id in users_to_include:
                if user_who_tweeted != screen_name:
                  #add an edge to the graph
                  if MEN_Graph.has_edge(user_who_tweeted, screen_name):
                    # If the edge exists, increment its weight
                    MEN_Graph[user_who_tweeted][screen_name]['weight'] += 1
                  else:
                    # If the edge doesn't exist, add it with weight=1
                    MEN_Graph.add_edge(user_who_tweeted, screen_name, weight=1)
print(f"{i} total tweets processed")

0 tweets iterated
10000 tweets iterated
20000 tweets iterated
30000 tweets iterated
40000 tweets iterated
50000 tweets iterated
60000 tweets iterated
70000 tweets iterated
80000 tweets iterated
90000 tweets iterated
100000 tweets iterated
110000 tweets iterated
120000 tweets iterated
130000 tweets iterated
140000 tweets iterated
150000 tweets iterated
160000 tweets iterated
170000 tweets iterated
175077 total tweets processed


In [31]:
graph_summary_stats(G = MEN_Graph)

----------------------------------------
##### Graph Summary #####
number of nodes: 194
number of edges: 339

nodes: ['kiganyi_', 'adidas', 'undefeatedinc', 'uniwatch', 'nike', 'atari_jones', 'adidasoriginals', 'solefed', 'jumpman23', 'bajabiri', 'golfdigest', 'lululemon', 'jonahlupton', 'nikestore', 'jermainedupri', 'finishline', 'wwd', 'hiphopwired', 'xboxwire', 'aarongreenberg', 'xbox', 'xboxp3', 'predsnhl', 'lakings', 'dashiexp', 'fastcompany', 'reignofapril', 'nrarmour', 'sbjsbd', 'barcaacademy', 'khou', 'oakley', 'marshablackburn', 'senrickscott', 'schuh', 'dezeen', 'lebatardshow', 'billiejeanking', 'barrysanders', 'complex', 'brooksrunning', 'adidasrunning', 'wiedenkennedy', 'bottom2thatop', 'candace_parker', 'snkr_twitr', 'katgraham', 'joshog', 'adweek', 'kingjames', 'jamesgunn', 'pomklementieff', 'loyalty360', 'jasonlacanfora', 'kohls', '7newsdc', 'realrclark25', 'adidashoops', 'wnba', 'barondavis', 'metropolismag', 'legiqn', 'fatkiddeals', 'jack_p', 'nyctsubway', 'nikebasketb

## Create Subgraphs for Analysis

In [33]:
##### Shared Sets of Node Intersetions #####

# Define key nodes
key_nodes_all = ['nike', 'lululemon', 'adidas']
key_nodes_nl = ['nike', 'lululemon']
key_nodes_na = ['nike', 'adidas']
key_nodes_al = ['adidas', 'lululemon']

# Find neighbors of the key nodes by themselves
neighbors_sets_n = set(nx.all_neighbors(MEN_Graph, 'nike'))
neighbors_sets_a = set(nx.all_neighbors(MEN_Graph, 'adidas'))
neighbors_sets_l = set(nx.all_neighbors(MEN_Graph, 'lululemon'))

# Find neighbors of the key nodes
neighbors_sets_all = [set(nx.all_neighbors(MEN_Graph, node)) for node in key_nodes_all]
neighbors_sets_nl = [set(nx.all_neighbors(MEN_Graph, node)) for node in key_nodes_nl]
neighbors_sets_na = [set(nx.all_neighbors(MEN_Graph, node)) for node in key_nodes_na]
neighbors_sets_al = [set(nx.all_neighbors(MEN_Graph, node)) for node in key_nodes_al]

# Intersect the sets to get nodes connected to all key nodes
common_neighbors_all = set.intersection(*neighbors_sets_all)

# Intersect the sets to get nodes connected to only 2 key nodes

common_neighbors_nl = set.intersection(*neighbors_sets_nl) - common_neighbors_all - set(key_nodes_al)
common_neighbors_na = set.intersection(*neighbors_sets_na) - common_neighbors_all - set(key_nodes_nl)
common_neighbors_al = set.intersection(*neighbors_sets_al) - common_neighbors_all - set(key_nodes_na)

# Get nodes connected to any one of the key nodes but not all of them
union_neighbors = set.union(*neighbors_sets_all)

# Get nodes connected to only 1 brand
exclusive_neighbors = (union_neighbors - common_neighbors_all
                       - common_neighbors_nl - common_neighbors_na - common_neighbors_al)

# Get nodes connected to each specific brand
exclusive_neighbors_n = neighbors_sets_n - common_neighbors_all - common_neighbors_nl - common_neighbors_na - set(key_nodes_al)
exclusive_neighbors_a = neighbors_sets_a - common_neighbors_all - common_neighbors_na - common_neighbors_nl - set(key_nodes_nl)
exclusive_neighbors_l = neighbors_sets_l - common_neighbors_all - common_neighbors_nl - common_neighbors_al - set(key_nodes_na)

In [34]:
##### Bridging to All Brands Nodes #####
# Create a subgraph
nodes_to_keep = list(common_neighbors_all) + key_nodes_all
MEN_Graph_bridge_3 = MEN_Graph.subgraph(nodes_to_keep)
#display a summary of the subgraph
graph_summary_stats(G = MEN_Graph_bridge_3, title='Bridging to All Brands Nodes')

##### Nike to Adidas Bridges #####
# Create a subgraph
nodes_to_keep = list(common_neighbors_na) + key_nodes_na
MEN_Graph_bridge_na = MEN_Graph.subgraph(nodes_to_keep)
#display a summary of the subgraph
graph_summary_stats(G = MEN_Graph_bridge_na, title='Nike to Adidas Bridges')

##### Nike to Lululemon Bridges #####
# Create a subgraph
nodes_to_keep = list(common_neighbors_nl) + key_nodes_nl
MEN_Graph_bridge_nl = MEN_Graph.subgraph(nodes_to_keep)
#display a summary of the subgraph
graph_summary_stats(G = MEN_Graph_bridge_nl, title='Nike to Lululemon Bridges')

##### Adidas to Lululemon Bridges #####
# Create a subgraph
nodes_to_keep = list(common_neighbors_al) + key_nodes_al
MEN_Graph_bridge_al = MEN_Graph.subgraph(nodes_to_keep)
#display a summary of the subgraph
graph_summary_stats(G = MEN_Graph_bridge_al, title='Adidas to Lululemon Bridges')

----------------------------------------
##### Bridging to All Brands Nodes #####
number of nodes: 6
number of edges: 10

nodes: ['adidas', 'lululemon', 'nike', 'deezefi', 'wwd', 'uniwatch']

neighbors of adidas: ['wwd', 'uniwatch', 'deezefi', 'nike']
neighbors of nike: ['uniwatch', 'wwd', 'deezefi', 'adidas']
neighbors of lululemon: ['uniwatch', 'deezefi', 'wwd']
----------------------------------------

----------------------------------------
##### Nike to Adidas Bridges #####
number of nodes: 29
number of edges: 57

nodes: ['hiphopwired', 'coindesk', 'burgerking', 'sbjsbd', 'schuh', 'kicksdeals', 'thebussypleaser', 'mattsteffanina', 'reebok', 'bajabiri', 'tonipayne', 'techinsider', 'wnba', 'tropofarmer', 'nicekicks', 'highsnobiety', 'finishline', 'cointelegraph', 'adidas', 'boredelonmusk', 'undefeatedinc', 'slamonline', 'complexstyle', 'snkr_twitr', 'xxxcrypt0', 'nike', 'dezeen', 'namecheap', 'jdofficial']

neighbors of adidas: ['undefeatedinc', 'bajabiri', 'hiphopwired', 'snkr_twi

In [35]:
##### Exclusive Nodes: Unique to Only 1 Brand #####

# Create a subgraph
nodes_to_keep_exc = list(exclusive_neighbors) + key_nodes_all
MEN_Graph_exc = MEN_Graph.subgraph(nodes_to_keep_exc)

#filter the graph edges to focus on influential to brands
#MEN_Graph_exc = focus_edges(G = MEN_Graph_exc, brand_nodes = key_nodes_all, weight_min = 3)

#display a summary of the subgraph
graph_summary_stats(G = MEN_Graph_exc, title='Exclusive Nodes: Unique to Only 1 Brand')

# Create a subgraph
MEN_Graph_exc_n = MEN_Graph.subgraph(list(exclusive_neighbors_n) + ['nike'])
MEN_Graph_exc_a = MEN_Graph.subgraph(list(exclusive_neighbors_a) + ['adidas'])
MEN_Graph_exc_l = MEN_Graph.subgraph(list(exclusive_neighbors_l) + ['lululemon'])

#filter the graph edges to focus on influential to brands
#MEN_Graph_exc_n = focus_edges(G = MEN_Graph_exc_n, brand_nodes = key_nodes_all, weight_min = 3)
#MEN_Graph_exc_a = focus_edges(G = MEN_Graph_exc_a, brand_nodes = key_nodes_all, weight_min = 3)

#display a summary of the subgraph
graph_summary_stats(G = MEN_Graph_exc_n, title='Exclusive Nodes for Nike')
graph_summary_stats(G = MEN_Graph_exc_a, title='Exclusive Nodes for Adidas')
graph_summary_stats(G = MEN_Graph_exc_l, title='Exclusive Nodes for Lululemon')

----------------------------------------
##### Exclusive Nodes: Unique to Only 1 Brand #####
number of nodes: 152
number of edges: 212

nodes: ['kiganyi_', 'adidas', 'nike', 'atari_jones', 'adidasoriginals', 'solefed', 'jumpman23', 'golfdigest', 'lululemon', 'jonahlupton', 'nikestore', 'jermainedupri', 'xboxwire', 'aarongreenberg', 'xbox', 'xboxp3', 'lakings', 'dashiexp', 'fastcompany', 'reignofapril', 'nrarmour', 'barcaacademy', 'oakley', 'marshablackburn', 'senrickscott', 'lebatardshow', 'billiejeanking', 'barrysanders', 'complex', 'adidasrunning', 'wiedenkennedy', 'bottom2thatop', 'candace_parker', 'katgraham', 'joshog', 'kingjames', 'jamesgunn', 'pomklementieff', 'loyalty360', 'jasonlacanfora', 'kohls', '7newsdc', 'adidashoops', 'barondavis', 'metropolismag', 'legiqn', 'fatkiddeals', 'jack_p', 'nyctsubway', 'nikebasketball', 'rga', 'stockx', 'gladstein', 'msmelchen', 'sproutsocial', 'orioles', 'natbrunell', 'fousey', 'okayplayer', 'sacramentokings', 'yahoofinance', 'enesfreedom', '

## Ploting All the Graphs

In [44]:
#plot the baseline graph
plot_graph(G = MEN_Graph, file_path='01_00_mention_network', plot_size='large')

MemoryError: In RendererAgg: Out of memory

Error in callback <function _draw_all_if_interactive at 0x000002100C67B6A0> (for post_execute), with arguments args (),kwargs {}:


MemoryError: In RendererAgg: Out of memory

MemoryError: In RendererAgg: Out of memory

<Figure size 30000x30000 with 1 Axes>

In [39]:
#plot the graph
plot_graph(G = MEN_Graph_bridge_3, file_path='02_01_mention_network_bridge_all_3', plot_size='small')

#plot the graph
plot_graph(G = MEN_Graph_bridge_na, file_path='02_02_mention_network_bridge_nike_adidas', plot_size='small')

#plot the graph
plot_graph(G = MEN_Graph_bridge_nl, file_path='02_03_mention_network_bridge_nike_lululemon', plot_size='small')

#plot the graph
plot_graph(G = MEN_Graph_bridge_al, file_path='02_04_mention_network_bridge_adidas_lululemon', plot_size='small')
     

In [40]:
#plot the exclusive node graph
plot_graph(G = MEN_Graph_exc, file_path='03_01_mention_network_exclusive', plot_size='medium')

#plot the graph
plot_graph(G = MEN_Graph_exc_n, file_path='03_02_mention_network_exclusive_to_nike', plot_size='small')

#plot the graph
plot_graph(G = MEN_Graph_exc_a, file_path='03_03_mention_network_exclusive_to_adidas', plot_size='small')

#plot the graph
plot_graph(G = MEN_Graph_exc_l, file_path='03_04_mention_network_exclusive_to_lulu', plot_size='small')

# Create a Semantic Network